In [ ]:
# standard imports
from typing import Dict, List
import numpy as np
import xarray as xr
import pandas as pd
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
with open('/Users/Adrian/Documents/VS CODE/spicy-snow/tests/test_data/2_img_ds', 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/Adrian/Documents/VS CODE/spicy-snow/spicy_snow')

# import the function to test
from processing.s1_preprocessing import s1_amp_to_dB, s1_dB_to_amp, merge_partial_s1_images

#dataset = ds.isel(time = slice(1, 3))
dataset = ds

In [ ]:
#dataset.attrs['s1_units'] = 'amp'

In [ ]:
dataset

In [ ]:
dataset.data_vars

In [ ]:
#s1_inc = s1[0, 2]
#s1_inc >= 0.7
#s1[dict(time=0, band=2)]

s1 = dataset.s1

### Plot Incidence Angles Before Mask

In [ ]:
# BEFORE

#s1_inc = s1[0, 2]
#s1_inc >= 0.7
#s1[dict(time=0, band=2)]

s1 = dataset.s1
s1_inc = s1.drop_sel(band=['VV', 'VH'])
s1_inc.plot()

### Test Masking Function

In [ ]:
# Select incidence angle from s1 data array
for band in ['inc']:
    data = dataset['s1'].sel(band=band)

    # Mask pixels with incidence angle > 70 degrees ('inc' <= 0.7)
    data_masked = data.where(data <= 0.7)

    dataset['s1'].loc[dict(band = band)] = data_masked

### Plot Incidence Angles After Mask

In [ ]:
# AFTER
s1_after = dataset.s1
s1_afterinc = s1_after.drop_sel(band=['VV', 'VH'])
s1_afterinc.plot()

## FINAL FUNCTION

In [ ]:
def s1_incidence_angle_masking(dataset: xr.Dataset, inplace: bool = False) -> xr.Dataset:
        """
        Remove s1 image outliers by masking pixels with incidence angles > 70 degrees

        Args:
        dataset: Xarray Dataset of sentinel images to mask incidence angle outliers

        Returns:
        dataset: Xarray Dataset of sentinel images with incidence angles > 70 degrees
        masked
        """

        # Check inplace flag
        if not inplace:
                dataset = dataset.copy(deep=True)

        # Select incidence angle from s1 data array
        # Mask pixel with incidence angles > 70 deg
        for band in ['inc']:
                data = dataset['s1'].sel(band=band)

                # Mask array with incidence angle > 70 degrees ('inc' <= 0.7)
                data_masked = data.where(data <= 0.7)

                dataset['s1'].loc[dict(band = band)] = data_masked

        if not inplace:
                return dataset